In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 로드

# 1. 곡 장르 코드 데이터
### 대분류 장르코드

In [403]:
genre_gn_all = pd.read_json('C:/Users/kjh96/Github/arena/genre_gn_all.json', typ = 'series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
genre_gn_all.head()

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00


In [399]:
# 장르코드 뒷자리 두 자리가 00인 코드를 필터링
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']
gnr_code.head()

,gnr_code,gnr_name
0,GN0100,발라드
6,GN0200,댄스
12,GN0300,랩/힙합
18,GN0400,R&B/Soul
22,GN0500,인디음악


### 상세 장르코드

In [406]:
# 장르코드 뒷자리 두 자리가 00이 아닌 코드를 필터링
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)
dtl_gnr_code.head()

C:\Users\kjh96\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,dtl_gnr_code,dtl_gnr_name
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
5,GN0105,'10-


### 장르 코드 트리

In [409]:
# 앞자리 네 자리 공통코드 추출
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# Merge
gnr_code_tree = pd.merge(gnr_code, dtl_gnr_code, how = 'left', on = 'join_code')

In [410]:
# 장르 최종
gnr_code_tree.isnull().sum()
gnr_code_tree[gnr_code_tree['dtl_gnr_code'].isnull()]

gnr_code_tree['dtl_gnr_code'][224] = ""
gnr_code_tree['dtl_gnr_name'][224] = ""

In [411]:
gnr_code_tree = gnr_code_tree.loc[gnr_code_tree['dtl_gnr_name'] != '세부장르전체']
gnr_code_tree = gnr_code_tree.reset_index(drop=True)

In [412]:
gnr_code_tree['genre_fin'] = gnr_code_tree['gnr_name'] + " " + gnr_code_tree['dtl_gnr_name']
gnr_code_tree = gnr_code_tree[['dtl_gnr_code','genre_fin' ]]
gnr_code_tree['dtl_gnr_code'][197] = 'GN3000'

In [413]:
# 매핑을 위해 딕셔너리화
gnr_code_tree = dict(zip(list(gnr_code_tree['dtl_gnr_code']) , list(gnr_code_tree['genre_fin'])))
gnr_code_tree

{'GN0102': "발라드 '80",
 'GN0103': "발라드 '90",
 'GN0104': "발라드 '00",
 'GN0105': "발라드 '10-",
 'GN0202': "댄스 '80",
 'GN0203': "댄스 '90",
 'GN0204': "댄스 '00",
 'GN0205': "댄스 '10-",
 'GN0302': '랩/힙합 랩 스타일',
 'GN0303': '랩/힙합 보컬 스타일',
 'GN0304': '랩/힙합 언더그라운드 힙합',
 'GN0305': '랩/힙합 시대별',
 'GN0402': 'R&B/Soul 어반',
 'GN0403': 'R&B/Soul R&B',
 'GN0502': '인디음악 포크',
 'GN0503': '인디음악 록',
 'GN0504': '인디음악 일렉',
 'GN0505': '인디음악 힙합',
 'GN0506': '인디음악 발라드',
 'GN0507': "인디음악 '90",
 'GN0508': "인디음악 '00",
 'GN0509': "인디음악 '10-",
 'GN0602': "록/메탈 '70",
 'GN0603': "록/메탈 '80",
 'GN0604': "록/메탈 '90",
 'GN0605': "록/메탈 '00",
 'GN0606': "록/메탈 '10-",
 'GN0702': '성인가요 신세대트로트',
 'GN0703': '성인가요 전설의트로트',
 'GN0704': '성인가요 뽕짝트로트',
 'GN0705': '성인가요 트로트메들리',
 'GN0706': "성인가요 트로트'60-'70",
 'GN0707': "성인가요 트로트'80-'90",
 'GN0708': "성인가요 트로트'00-",
 'GN0709': "성인가요 성인가요'80-'90",
 'GN0710': "성인가요 성인가요'00-",
 'GN0802': "포크/블루스 '60-'70",
 'GN0803': "포크/블루스 '80-'90",
 'GN0804': "포크/블루스 '00",
 'GN0805': "포크/블루스 '10-",
 'GN0902': 'POP 

# 2. 노래 메타 데이터

In [6]:
song_meta = pd.read_json('C:/Users/kjh96/Github/arena/song_meta.json')

# 3. 플레이리스트 Train data

In [414]:
train = pd.read_json('C:/Users/kjh96/Github/arena/train.json')
train = train[:200]
train_df = train[['plylst_title','songs','tags']]

In [415]:
train_df.head()

,plylst_title,songs,tags
0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락]
1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]"
2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]"
3,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,..."
4,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스]


In [416]:
list_df = []
for i in range(len(train_df)) :
    df = pd.DataFrame(train_df['songs'][i])
    df = df.assign(tags = " ".join(train_df['tags'][i]),plylst_title = train_df['plylst_title'][i])
    list_df.append(df)

In [417]:
for i in range(len(list_df)) : 
    if i == 0 :    
        df_fin = list_df[0]
    else :
        df_fin = pd.concat([df_fin,list_df[i]])

In [418]:
df_fin = df_fin.rename(columns={0 : 'song_id'})
song_meta = song_meta.rename(columns={'id' : 'song_id'})

# 노래 메타 정보와 merge
df_fin = pd.merge(df_fin, song_meta, how = 'left', on = "song_id")

# 콘텐츠 기반 필터링 추천(Content based filtering)

In [326]:
data = df_fin
data.head()

,song_id,tags,plylst_title,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
0,525514,락,여행같은 음악,"[GN1402, GN1401]",20130506,Hey Little Girl,2200223,[734201],Hey Little Girl,[GN1400],[The Sol]
1,129701,락,여행같은 음악,"[GN0901, GN0902, GN1001]",20130917,Brass,2201802,[536907],Octagon,"[GN0900, GN1000]",[Royal Bangs]
2,383374,락,여행같은 음악,"[GN1012, GN1005, GN1001]",19911021,Monsters Under The Bed,2216938,[166978],The Road,[GN1000],[Honeymoon Suite]
3,562083,락,여행같은 음악,"[GN1013, GN0901, GN0902, GN1001]",20000919,United,43227,[19035],Honeymoon,"[GN0900, GN1000]",[Phoenix]
4,297861,락,여행같은 음악,"[GN1013, GN0901, GN0902, GN1001]",20050306,Back To Bedlam,303657,[170117],High,"[GN0900, GN1000]",[James Blunt]


# 매핑

In [327]:
data['genre'] = ""
for i in range(len(data)) :
    data['genre'][i] = list()
    for j in range(len(data['song_gn_dtl_gnr_basket'][i])) : 
        if data['song_gn_dtl_gnr_basket'][i][j] in gnr_code_tree : 
            data['genre'][i].append(gnr_code_tree.get(data['song_gn_dtl_gnr_basket'][i][j]))
    data['genre'][i] = " ".join(data['genre'][i])

C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [352]:
count_vector = CountVectorizer(ngram_range=(1, 1))
c_vector_genres = count_vector.fit_transform(data['genre'])
c_vector_genres.shape

(8038, 119)

In [353]:
#코사인 유사도를 구한 벡터를 미리 저장
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]
gerne_c_sim.shape
gerne_c_sim

(8038, 8038)

In [423]:
def get_recommend_song_list(df, song_id, top=30):
    # 특정 노래와 비슷한 노래를 추천해야 하기 때문에 '특정 노래' 정보를 뽑아낸다.
    target_song_index = df[df['song_id'] == song_id].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_song_index, :top].reshape(-1)
    
    #본인을 제외
    sim_index = sim_index[sim_index != target_song_index]
    sim_index = sim_index[0]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('issue_date', ascending=False)[:10]
    return result

In [426]:
get_recommend_song_list(data, song_id= 6546)

C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


,song_id,tags,plylst_title,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,genre
4530,121860,비오는날 감성 비 아련 발라드 멜로 OST 치명 사랑,치명적이고도 치정적 사랑의 OST,"[GN0105, GN1501, GN0101, GN1504]",20200328,부부의 세계 OST Part.1,10409542,[1767],고독한 항해,"[GN1500, GN0100]",[김윤아],발라드 '10- OST 국내드라마
4530,121860,비오는날 감성 비 아련 발라드 멜로 OST 치명 사랑,치명적이고도 치정적 사랑의 OST,"[GN0105, GN1501, GN0101, GN1504]",20200328,부부의 세계 OST Part.1,10409542,[1767],고독한 항해,"[GN1500, GN0100]",[김윤아],발라드 '10- OST 국내드라마
4530,121860,비오는날 감성 비 아련 발라드 멜로 OST 치명 사랑,치명적이고도 치정적 사랑의 OST,"[GN0105, GN1501, GN0101, GN1504]",20200328,부부의 세계 OST Part.1,10409542,[1767],고독한 항해,"[GN1500, GN0100]",[김윤아],발라드 '10- OST 국내드라마
803,331764,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170114,불어라 미풍아 OST Part.18,10031388,[757478],반을 잃었다,"[GN1500, GN0100]",[공기남녀],발라드 '10- OST 국내드라마
803,331764,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170114,불어라 미풍아 OST Part.18,10031388,[757478],반을 잃었다,"[GN1500, GN0100]",[공기남녀],발라드 '10- OST 국내드라마
803,331764,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170114,불어라 미풍아 OST Part.18,10031388,[757478],반을 잃었다,"[GN1500, GN0100]",[공기남녀],발라드 '10- OST 국내드라마
802,18960,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170108,도깨비 OST Part.10,10029325,[420527],소원,"[GN1500, GN0100]",[어반자카파],발라드 '10- OST 국내드라마
802,18960,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170108,도깨비 OST Part.10,10029325,[420527],소원,"[GN1500, GN0100]",[어반자카파],발라드 '10- OST 국내드라마
802,18960,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170108,도깨비 OST Part.10,10029325,[420527],소원,"[GN1500, GN0100]",[어반자카파],발라드 '10- OST 국내드라마
801,224921,조용히 혼자 또는 새벽감성 고민 맥주한잔,조용히 맥주 한잔 할때_새벽감성,"[GN0105, GN1501, GN0101, GN1504]",20170107,도깨비 OST Part.9,10029181,[629831],첫눈처럼 너에게 가겠다,"[GN1500, GN0100]",[에일리],발라드 '10- OST 국내드라마


In [425]:
song_meta[song_meta['song_id']==6546]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,song_id
6546,"[GN0105, GN1501, GN0101, GN1504]",20140902,연애의 발견 OST Part 4,2279896,[192827],"묘해, 너와","[GN1500, GN0100]",[어쿠스틱 콜라보],6546
